# Software Enginneering Project

In [4]:
import oracledb

username = "login_user"
password = "login_password"
dsn = "localhost:1521/ORCLPDB1"

try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Error:", e)


email='admin@school.com'
password='hashed_password1'

cursor = connection.cursor()
query = """
            SELECT role FROM users 
            WHERE email = :email AND password = :password
            """

#query=""" SELECT * FROM USERS"""
#query="""SELECT table_name FROM user_tables"""
#query= """SELECT USER FROM DUAL"""

cursor.execute(query, {"email": email, "password": password})
#cursor.execute(query)
result = cursor.fetchone()
print(f"Query Result -> {result}")






Connection successful!
Query Result -> ('admin',)


In [2]:
import oracledb
import pandas as pd
import streamlit as st


def connect_to_db():
    """Connect to the Oracle database."""
    try:
        connection = oracledb.connect(
            user = "login_user",
            password = "login_password",
            dsn = "localhost:1521/ORCLPDB1"
        )
        
        return connection
    except oracledb.DatabaseError as e:
        print("Database connection failed:", e)
        return None
    


def view_courses():
    """
    Fetch all courses from the courses table.

    Returns:
        rows (list of tuples): A list of all course records.
        columns (list): A list of column names corresponding to the records.
    """
    connection = connect_to_db()  # Remplacez par votre fonction de connexion
    if connection:
        cursor = connection.cursor()
        try:
            query = """
            SELECT course_id, course_name, description, credit_hours, department,
                   teacher_id, schedule, semester, max_students, created_at, updated_at
            FROM courses
            """
            cursor.execute(query)
            rows = cursor.fetchall()  
            columns = [col[0] for col in cursor.description]  
            return rows, columns
        except oracledb.DatabaseError as e:
            print(f"Failed to fetch courses: {e}")
            return None, None
        finally:
            cursor.close()  # Toujours fermer le curseur
            connection.close()  # Toujours fermer la connexion
    else:
        print("Database connection could not be established.")
        return None, None


view_courses()

([(3,
   'Advanced Machine Learning',
   4,
   'Computer Science',
   22,
   'Tue, Thu 2:00 PM - 4:00 PM',
   'Spring 2024',
   30,
   datetime.datetime(2024, 12, 13, 22, 7, 25, 374398),
   datetime.datetime(2024, 12, 13, 22, 7, 25, 374398))],
 ['COURSE_ID',
  'COURSE_NAME',
  'DESCRIPTION',
  'CREDIT_HOURS',
  'DEPARTMENT',
  'TEACHER_ID',
  'SCHEDULE',
  'SEMESTER',
  'MAX_STUDENTS',
  'CREATED_AT',
  'UPDATED_AT'])

In [3]:
def connect_to_db():
    """Connect to the Oracle database."""
    try:
        connection = oracledb.connect(
            user = "login_user",
            password = "login_password",
            dsn = "localhost:1521/ORCLPDB1"
        )
        
        return connection
    except oracledb.DatabaseError as e:
        print("Database connection failed:", e)
        return None
    
connect_to_db()

<oracledb.Connection to login_user@localhost:1521/ORCLPDB1>

In [5]:
def add_student_to_course(student_id, course_id):
    """
    Add a student to a course in the student_courses table.

    Args:
        student_id (int): The ID of the student.
        course_id (int): The ID of the course.

    Returns:
        bool: True if the operation was successful, False otherwise.
    """
    connection = connect_to_db()  # Ensure this connects to your database
    if connection:
        try:
            cursor = connection.cursor()
            query = """
            INSERT INTO student_courses (student_id, course_id, enrollment_date)
            VALUES (:1, :2, CURRENT_TIMESTAMP)
            """
            cursor.execute(query, [student_id, course_id])
            connection.commit()
            cursor.close()
            connection.close()
            return True
        except oracledb.DatabaseError as e:
            print(f"Failed to add student {student_id} to course {course_id}: {e}")
            return False
    else:
        print("Database connection could not be established.")
        return False


add_student_to_course(1,2)

True

In [2]:

class DatabaseConnection:
    _instance = None  # Static variable to hold the single instance

    @staticmethod
    def get_connection():
        """Static access method to get the database connection."""
        try:
            # Check if the connection exists and is still open
            if DatabaseConnection._instance is None or DatabaseConnection._instance.ping() is not None:
                # Create or reconnect the instance
                DatabaseConnection._instance = oracledb.connect(
                    user="login_user",
                    password="login_password",
                    dsn="localhost:1521/ORCLPDB1"
                )
        except oracledb.DatabaseError as e:
            print("Database connection failed:", e)
            DatabaseConnection._instance = None
        return DatabaseConnection._instance

    @staticmethod
    def close_connection():
        """Close the database connection."""
        if DatabaseConnection._instance is not None:
            try:
                DatabaseConnection._instance.close()
                DatabaseConnection._instance = None
                print("Database connection closed successfully.")
            except oracledb.DatabaseError as e:
                print("Failed to close the database connection:", e)

In [7]:
print(DatabaseConnection.get_connection())

<oracledb.Connection to login_user@localhost:1521/ORCLPDB1>


In [5]:
def get_courses_by_teacher_id(teacher_id):
    """
    Retrieve all courses taught by the teacher with the given teacher_id.
    """
    connection = DatabaseConnection.get_connection()
    if not connection:
        print("Database connection is not established.")
        return None

    try:
        with connection.cursor() as cursor:
            # Define the query
            query = """
                SELECT 
                    course_id, course_name, description, credit_hours, department, 
                    day_of_week, start_time, end_time, semester
                FROM courses
                WHERE teacher_id = :teacher_id
            """
            # Execute the query
            cursor.execute(query, {"teacher_id": teacher_id})

            # Fetch all results
            rows = cursor.fetchall()

            # If no results are found, return an empty list
            if not rows:
                return []

            # Map the results to a list of dictionaries
            courses = []
            for row in rows:
                courses.append({
                    "course_id": row[0],
                    "course_name": row[1],
                    "description": row[2].read() if row[2] else None,
                    "credit_hours": row[3],
                    "department": row[4],
                    "day_of_week": row[5],
                    "start_time": row[6],
                    "end_time": row[7],
                    "semester": row[8],
                })
            return courses

    except oracledb.Error as e:
        print("Error retrieving courses:", e)
        return None

get_courses_by_teacher_id(22)

[{'course_id': 3,
  'course_name': 'english',
  'description': 'e',
  'credit_hours': 5,
  'department': 'e',
  'day_of_week': 'Wednesday',
  'start_time': datetime.datetime(2024, 12, 14, 12, 14),
  'end_time': datetime.datetime(2024, 12, 14, 18, 0),
  'semester': '2'},
 {'course_id': 4,
  'course_name': 'fr',
  'description': 'z',
  'credit_hours': 1,
  'department': 'd',
  'day_of_week': 'Monday',
  'start_time': datetime.datetime(2024, 12, 14, 5, 45),
  'end_time': datetime.datetime(2024, 12, 14, 6, 45),
  'semester': '3'},
 {'course_id': 5,
  'course_name': 'info',
  'description': 'info',
  'credit_hours': 1,
  'department': 'info',
  'day_of_week': 'Monday',
  'start_time': datetime.datetime(2024, 12, 14, 14, 13),
  'end_time': datetime.datetime(2024, 12, 14, 15, 15),
  'semester': '2'},
 {'course_id': 2,
  'course_name': 'physics',
  'description': 'p',
  'credit_hours': 3,
  'department': 'phys',
  'day_of_week': 'Tuesday',
  'start_time': datetime.datetime(2024, 12, 14, 15, 0)

In [7]:
def get_teacher_id_by_user_id(user_id):
    """
    Retrieve the teacher_id associated with the given user_id from the database.
    """
    connection = DatabaseConnection.get_connection()
    if not connection:
        print("Database connection is not established.")
        return None

    try:
        with connection.cursor() as cursor:
            # Define the query
            query = """
                SELECT teacher_id
                FROM teachers
                WHERE user_id = :user_id
            """
            # Execute the query
            cursor.execute(query, {"user_id": user_id})

            # Fetch the result
            row = cursor.fetchone()

            # If no result is found, return None
            if not row:
                return None

            # Return the teacher_id
            return row[0]

    except oracledb.Error as e:
        print("Error retrieving teacher_id:", e)
        return None



print(get_teacher_id_by_user_id(40))

22
